In [2]:
%pwd

'/orcd/data/edboyden/002/ezh/uni'

In [3]:
import pandas as pd

In [26]:
df = pd.read_csv("coadread_tcga_clinical_data.tsv", sep="\t")

# Save it as a CSV file
df.to_csv("coadread_tcga_clinical_data.csv", index=False)

In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('coadread_tcga_clinical_data.csv')  # Replace with your actual file path

df.head()

,SAMPLE_ID,ERBB2,HER2_Amp
0,TCGA-F5-6861-01,4736.6678,0
1,TCGA-EI-6509-01,511917.4691,1
2,TCGA-F5-6864-01,7501.1355,1
3,TCGA-DC-6155-01,2457.5163,0
4,TCGA-AH-6547-01,4242.6181,0


In [7]:
df = df.rename(columns={"SAMPLE ID": "Sample_ID"})

In [8]:
print(df.columns)

Index(['Sample_ID', 'ERBB2', 'HER2_Amp'], dtype='object')


In [30]:

df = df[['Sample_ID', "Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code"]]  # Keep only these columns
print(df.head(10))

         Sample_ID  \
0  TCGA-3L-AA1B-01   
1  TCGA-4N-A93T-01   
2  TCGA-4T-AA8H-01   
3  TCGA-5M-AAT4-01   
4  TCGA-5M-AAT5-01   
5  TCGA-5M-AAT6-01   
6  TCGA-5M-AATA-01   
7  TCGA-5M-AATE-01   
8  TCGA-A6-2670-01   
9  TCGA-A6-2671-01   

  Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code  
0                                                 N0                         
1                                                N1b                         
2                                                 N0                         
3                                                 N0                         
4                                                NaN                         
5                                                N2b                         
6                                                NaN                         
7                                                 N0                         
8                                                 N0   

In [9]:
# remove the -01
df['Sample_ID'] = df['Sample_ID'].str.replace(r'-01$', '', regex=True)
print(df.head(10))

      Sample_ID        ERBB2  HER2_Amp
0  TCGA-F5-6861    4736.6678         0
1  TCGA-EI-6509  511917.4691         1
2  TCGA-F5-6864    7501.1355         1
3  TCGA-DC-6155    2457.5163         0
4  TCGA-AH-6547    4242.6181         0
5  TCGA-AF-2687    2646.6582         0
6  TCGA-EI-6513    4822.8883         1
7  TCGA-AF-6655    2812.6769         0
8  TCGA-DC-6157    5318.3408         1
9  TCGA-AF-A56K    6662.3084         1


In [20]:
# Create a new binary BRAF column
# Initialize all values to 0
# df['BRAF'] = 0

df.loc[df['Perineural Invasion'].str.contains('YES', case=False, na=False), 'Perineural Invasion'] = 1
df.loc[df['Perineural Invasion'].str.contains('NO', case=False, na=False), 'Perineural Invasion'] = 0

# Display the first few rows to verify

df = df[['Sample_ID', "Perineural Invasion"]]  # Keep only these columns
print(df.head(10))

      Sample_ID Perineural Invasion
0  TCGA-3L-AA1B                   0
1  TCGA-4N-A93T                   0
2  TCGA-4T-AA8H                   0
3  TCGA-5M-AAT4                 NaN
4  TCGA-5M-AAT5                 NaN
5  TCGA-5M-AAT6                   1
6  TCGA-5M-AATA                 NaN
7  TCGA-5M-AATE                   0
8  TCGA-A6-2670                 NaN
9  TCGA-A6-2671                 NaN


In [10]:
# Save the modified dataframe to a new CSV
df.to_csv('perineural_invasion.csv', index=False)

In [18]:
import os
import h5py
import pandas as pd

# Path to folder containing H5 files
h5_dir = "CLAM/TCGA-COAD_h5/"
results = []

# Loop through all .h5 files in directory
for fname in os.listdir(h5_dir):
    if fname.endswith(".h5"):
        # print("here")
        h5_path = os.path.join(h5_dir, fname)
        sample_id = fname.replace(".h5", "")

        try:
            with h5py.File(h5_path, "r") as f:
                if "mmr_status" in f:
                    
                    msi_value = f["mmr_status"][()]
                    if isinstance(msi_value, bytes):  # in case it's stored as a byte string
                        msi_value = msi_value.decode()

                    # Some values might be arrays, so handle that
                    if isinstance(msi_value, (list, tuple)) or hasattr(msi_value, "__len__"):
                        msi_value = msi_value[0]

                    # Filter for binary 0 or 1
                    if msi_value in [0, 1, '0', '1', 0.0, 1.0]:
                        results.append({
                            "Sample_ID": sample_id,
                            "mmr_status": int(msi_value)
                        })

        except Exception as e:
            print(f"Failed to read {h5_path}: {e}")

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("msi_status.csv", index=False)


In [2]:
import pandas as pd

# Load the CSV (change path as needed)
df = pd.read_csv("magnifications.csv")

# Extract sample ID (first 3 parts of the slide name)
def extract_sample_id(slide_name):
    if isinstance(slide_name, str) and "DX1" in slide_name:
        parts = slide_name.split("-")
        return "-".join(parts[:3])
    return None  # or keep as is

# Create new column
df["Sample_ID"] = df["slide"].apply(extract_sample_id)

# Save or inspect
df.to_csv("magnifications.csv", index=False)
df.head()

,slide,magnification,Sample_ID
0,TCGA-F4-6463-01Z-00-DX1.a3fa6fb4-ce9d-4f0d-b5f...,40.0,TCGA-F4-6463
1,TCGA-AA-3561-01Z-00-DX1.1b5a2925-53f9-470f-a62...,20.0,TCGA-AA-3561
2,TCGA-AA-A02E-01Z-00-DX1.04D47621-9DCF-437C-A4D...,20.0,TCGA-AA-A02E
3,TCGA-AA-A01C-01Z-00-DX1.C69C8FC3-04A2-46B8-857...,20.0,TCGA-AA-A01C
4,TCGA-AZ-6607-01Z-00-DX1.b0a25161-7e13-42d6-927...,40.0,TCGA-AZ-6607
